In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("kaggle_dataset\marketing_campaign\marketing_campaign.csv", sep=";")

df.head()

<>:4: SyntaxWarning: invalid escape sequence '\m'
<>:4: SyntaxWarning: invalid escape sequence '\m'
C:\Users\bhc02\AppData\Local\Temp\ipykernel_1876\784271380.py:4: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv("kaggle_dataset\marketing_campaign\marketing_campaign.csv", sep=";")


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [2]:
#1. 미사용 컬럼 제거
df = df.drop(columns=['Education', 'Marital_Status', 'Dt_Customer', 'Recency', 'Z_CostContact', 'Z_Revenue'])
df.head()

,ID,Year_Birth,Income,Kidhome,Teenhome,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,58138.0,0,0,635,88,546,172,88,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,46344.0,1,1,11,1,6,2,1,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,71613.0,0,0,426,49,127,111,21,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,26646.0,1,0,11,4,20,10,3,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,58293.0,1,0,173,43,118,46,27,...,3,6,5,0,0,0,0,0,0,0


In [3]:
#2. 결측치 처리
df.isna().sum()
    #Income 결측치가 24개 존재
    #-> 제거로 처리
df = df.dropna()
df.isna().sum()

ID                     0
Year_Birth             0
Income                 0
Kidhome                0
Teenhome               0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
dtype: int64

In [4]:
#3. 이상치 처리

# 나이
age_mask = (df['Year_Birth'] > 1930) & (df['Year_Birth'] < 2010)
df = df[age_mask]
# 소득
income_mask = (df['Income'] > 0) & (df['Income'] < 200000)
df = df[income_mask]

In [9]:
#4. 파생변수 생성
    #컬럼으로 추가 : 영구 파생변수 / 변수로만 사용 : 일시 파생변수
    #나이
df['Age'] = 2020 - df['Year_Birth']
    #총 캠페인 반응 횟수
total_cmp = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
df['total_accepted_cmp'] = df[total_cmp].sum(axis=1)

#4.5. (인코딩)
    #문자를 숫자로 바꾸는 작업
    #순서가 있는 데이터 경우 0~n까지 숫자로 매핑 : 라벨인코딩
    #순서가 없는 데이터 경우 더미변수화 : 원핫인코딩
         #ex) single, married, toghether컬럼을 하나씩 만들고 해당되는 칸에 1표시

604


In [10]:
#5. 정규화
    #수입 정규화_standard
mean_income = df['Income'].mean()
std_income = df['Income'].std()

standard_income = (df['Income'] - mean_income) / std_income

In [ ]:
# 단순회귀 - 나이별 캠페인 반응 / 소득별 캠페인 반응 / 
# 다중회귀 - 특정 상품구매횟수를 늘린 캠페인이 있는가? / 

#!통제변수 없이 진행

#6. 회귀
    
    #(단순회귀1. 캠페인에 1번 이상 반응한 고객은 오프라인 구매를 더 많이 할까?)
#종속변수 Y : NumWebPurchases
#독립변수 X : total_accepted_cmp

df['CampaignResponse'] = (df['total_accepted_cmp'] > 0).astype(int)
df.groupby('CampaignResponse')['NumWebPurchases'].mean()

''